In [ ]:
import tensorflow as tf  
from keras.preprocessing.image import ImageDataGenerator  
from keras.regularizers import l2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization

In [26]:
tf.__version__  

'2.5.0'

In [27]:
train_datagen = ImageDataGenerator(rescale = 1./255,  
                                   shear_range = 0.2,  
                                   zoom_range = 0.2,  
                                   horizontal_flip = True)  



In [28]:
training_set = train_datagen.flow_from_directory('dataset/training_set',  
                                                 target_size = (64, 64),  
                                                 batch_size = 32,  
                                                 class_mode = 'categorical')  

Found 900 images belonging to 3 classes.


In [29]:
test_datagen = ImageDataGenerator(rescale = 1./255)  


In [30]:
test_set = test_datagen.flow_from_directory('dataset/testing_set',  
                                            target_size = (64, 64),  
                                            batch_size = 32,  
                                            class_mode = 'categorical')  

Found 75 images belonging to 3 classes.


In [31]:
model = tf.keras.models.Sequential()  

In [32]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))  


In [33]:
l2_reg = 0

# 1st Conv Layer : CRP 
model.add(Conv2D(96, (11, 11), input_shape=(64,64,3),
    padding='same', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Conv Layer 
model.add(Conv2D(256, (5, 5),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3,3),  padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 4th Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024,(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# 5th Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024,  (3 , 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

# 1st FC Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd FC Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 3rd FC Layer
model.add(Dense(3))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# print(model.summary())

model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])

In [34]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
module_wrapper_34 (ModuleWra (None, 62, 62, 96)        371808    
_________________________________________________________________
module_wrapper_35 (ModuleWra (None, 62, 62, 96)        384       
_________________________________________________________________
module_wrapper_36 (ModuleWra (None, 62, 62, 96)        0         
_________________________________________________________________
module_wrapper_37 (ModuleWra (None, 31, 31, 96)        0         
_________________________________________________________________
module_wrapper_38 (ModuleWra (None, 31, 31, 256)       614656    
_________________________________________________________________
module_wrapper_39 (ModuleWra (None, 31, 31, 256)      

In [ ]:
model.fit(x = training_set, validation_data = test_set, epochs = 75)  